In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [34]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page, attrs={'class':'wikitable'})
print('Extracted {num} wikitables'.format(num=len(wikitables)))
wikitables

Extracted1 wikitables


[    Postal code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                           Neighborhood  
 0                                                  NaN  
 1                                                  NaN  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                           Regent Park / Harbourfront  
 ..                                                 ...  
 175                                                NaN  
 176                                                NaN  
 177                

In [48]:
table = wikitables[0]
table = pd.DataFrame(table)
table

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [56]:
table2 = table.drop(table[table['Borough'] == 'Not assigned'].index)
table2 = table2.rename(columns={"Postal code": "PostalCode"})
table2

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [106]:
table3 = table2.groupby(by=['PostalCode'], sort=False).agg( ','.join)
table3.astype(str)
table3.reset_index()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [107]:
table3['Neighborhood'] = table3['Neighborhood'].str.replace(" / ",", ")
table3

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,Business reply mail Processing CentrE


In [112]:
table3.reset_index()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [113]:
table3.shape

(103, 2)

Looking through the data, there were no cases where there was a neighborhood and not a borough, so that step was eliminated from this code.  Everything else should be here.

In [114]:
latlong = pd.read_csv('http://cocl.us/Geospatial_data')
latlong.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [116]:
latlong = latlong.rename(columns={"Postal Code": "PostalCode"})
latlong

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [117]:
mergedDf = table3.merge(latlong, on='PostalCode')
mergedDf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [137]:
mergedDf3 = mergedDf.groupby(by=['Borough']).agg( ','.join)

In [138]:
mergedDf3

,PostalCode,Neighborhood
Borough,,
Central Toronto,"M4N,M5N,M4P,M5P,M4R,M5R,M4S,M4T,M4V","Lawrence Park,Roselawn,Davisville North,Forest..."
Downtown Toronto,"M5A,M7A,M5B,M5C,M5E,M5G,M6G,M5H,M5J,M5K,M5L,M5...","Regent Park, Harbourfront,Queen's Park, Ontari..."
East Toronto,"M4E,M4K,M4L,M4M,M7Y","The Beaches,The Danforth West, Riverdale,India..."
East York,"M4B,M4C,M4G,M4H,M4J","Parkview Hill, Woodbine Gardens,Woodbine Heigh..."
Etobicoke,"M9A,M9B,M9C,M9P,M9R,M8V,M9V,M8W,M9W,M8X,M8Y,M8Z","Islington Avenue,West Deane Park, Princess Gar..."
Mississauga,M7R,Canada Post Gateway Processing Centre
North York,"M3A,M4A,M6A,M3B,M6B,M3C,M2H,M3H,M2J,M3J,M2K,M3...","Parkwoods,Victoria Village,Lawrence Manor, Law..."
Scarborough,"M1B,M1C,M1E,M1G,M1H,M1J,M1K,M1L,M1M,M1N,M1P,M1...","Malvern, Rouge,Rouge Hill, Port Union, Highlan..."
West Toronto,"M6H,M6J,M6K,M6P,M6R,M6S","Dufferin, Dovercourt Village,Little Portugal, ..."


In [143]:
mergedDf4 = mergedDf3.reset_index()
mergedDf4

,Borough,PostalCode,Neighborhood
0,Central Toronto,"M4N,M5N,M4P,M5P,M4R,M5R,M4S,M4T,M4V","Lawrence Park,Roselawn,Davisville North,Forest..."
1,Downtown Toronto,"M5A,M7A,M5B,M5C,M5E,M5G,M6G,M5H,M5J,M5K,M5L,M5...","Regent Park, Harbourfront,Queen's Park, Ontari..."
2,East Toronto,"M4E,M4K,M4L,M4M,M7Y","The Beaches,The Danforth West, Riverdale,India..."
3,East York,"M4B,M4C,M4G,M4H,M4J","Parkview Hill, Woodbine Gardens,Woodbine Heigh..."
4,Etobicoke,"M9A,M9B,M9C,M9P,M9R,M8V,M9V,M8W,M9W,M8X,M8Y,M8Z","Islington Avenue,West Deane Park, Princess Gar..."
5,Mississauga,M7R,Canada Post Gateway Processing Centre
6,North York,"M3A,M4A,M6A,M3B,M6B,M3C,M2H,M3H,M2J,M3J,M2K,M3...","Parkwoods,Victoria Village,Lawrence Manor, Law..."
7,Scarborough,"M1B,M1C,M1E,M1G,M1H,M1J,M1K,M1L,M1M,M1N,M1P,M1...","Malvern, Rouge,Rouge Hill, Port Union, Highlan..."
8,West Toronto,"M6H,M6J,M6K,M6P,M6R,M6S","Dufferin, Dovercourt Village,Little Portugal, ..."
9,York,"M6C,M6E,M6M,M6N,M9N","Humewood-Cedarvale,Caledonia-Fairbanks,Del Ray..."


In [146]:
boroughcords = mergedDf[['Borough', 'Latitude', 'Longitude']]
boroughcords = boroughcords.groupby(by = ['Borough']).mean()
boroughcords.reset_index()

,Borough,Latitude,Longitude
0,Central Toronto,43.701980,-79.398954
1,Downtown Toronto,43.654597,-79.383972
2,East Toronto,43.669436,-79.324654
3,East York,43.700303,-79.335851
4,Etobicoke,43.660043,-79.542074
5,Mississauga,43.636966,-79.615819
6,North York,43.750727,-79.429338
7,Scarborough,43.766229,-79.249085
8,West Toronto,43.652653,-79.449290
9,York,43.690797,-79.472633


In [147]:
consoltable = mergedDf4.merge(boroughcords, on='Borough')
consoltable

,Borough,PostalCode,Neighborhood,Latitude,Longitude
0,Central Toronto,"M4N,M5N,M4P,M5P,M4R,M5R,M4S,M4T,M4V","Lawrence Park,Roselawn,Davisville North,Forest...",43.701980,-79.398954
1,Downtown Toronto,"M5A,M7A,M5B,M5C,M5E,M5G,M6G,M5H,M5J,M5K,M5L,M5...","Regent Park, Harbourfront,Queen's Park, Ontari...",43.654597,-79.383972
2,East Toronto,"M4E,M4K,M4L,M4M,M7Y","The Beaches,The Danforth West, Riverdale,India...",43.669436,-79.324654
3,East York,"M4B,M4C,M4G,M4H,M4J","Parkview Hill, Woodbine Gardens,Woodbine Heigh...",43.700303,-79.335851
4,Etobicoke,"M9A,M9B,M9C,M9P,M9R,M8V,M9V,M8W,M9W,M8X,M8Y,M8Z","Islington Avenue,West Deane Park, Princess Gar...",43.660043,-79.542074
5,Mississauga,M7R,Canada Post Gateway Processing Centre,43.636966,-79.615819
6,North York,"M3A,M4A,M6A,M3B,M6B,M3C,M2H,M3H,M2J,M3J,M2K,M3...","Parkwoods,Victoria Village,Lawrence Manor, Law...",43.750727,-79.429338
7,Scarborough,"M1B,M1C,M1E,M1G,M1H,M1J,M1K,M1L,M1M,M1N,M1P,M1...","Malvern, Rouge,Rouge Hill, Port Union, Highlan...",43.766229,-79.249085
8,West Toronto,"M6H,M6J,M6K,M6P,M6R,M6S","Dufferin, Dovercourt Village,Little Portugal, ...",43.652653,-79.449290
9,York,"M6C,M6E,M6M,M6N,M9N","Humewood-Cedarvale,Caledonia-Fairbanks,Del Ray...",43.690797,-79.472633


In [151]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [183]:
map_points = folium.Map(location=[43.6532, -79.3832], zoom_start=11)
total_points = 10
# set color scheme for the clusters
x = np.arange(total_points)
ys = [i + x + (i*x)**2 for i in range(total_points)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
c=0

# add markers to the map
markers_colors = []
for lat, lon, borough in zip(consoltable['Latitude'], consoltable['Longitude'], consoltable['Borough']):
    label = folium.Popup(str(borough), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[c], fill=True, fill_color=rainbow[c], fill_opacity=0.7).add_to(map_points)
    c=c
       
map_points

I grouped the points into the corresponding boroughs.  I took the average latitude and longitude point for each burough and placed a label of the correspoinding borough at that point.